In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import model_from_json

In [ ]:
print(os.listdir('../own_dataset/diferent_dice/train'))

In [ ]:
train_path= ('../own_dataset/diferent_dice/train')
test_path= ('../own_dataset/diferent_dice/test')

In [ ]:
batch_size_train=30  #,batch_size= batch_size_train
batch_size_valid=30
targetsize= 50
datagen=ImageDataGenerator(rotation_range=180,height_shift_range=0.1,rescale=1.0/255.0
                           ,zoom_range=[0.7,1.1],brightness_range=[0.4,1.2])
train_batches= datagen.flow_from_directory(train_path, target_size=(targetsize,targetsize), 
                              classes=os.listdir('../own_dataset/diferent_dice/train'),
                              batch_size= batch_size_train)
test_batches= datagen.flow_from_directory(test_path,  target_size=(targetsize,targetsize), 
                              classes=os.listdir('../own_dataset/diferent_dice/train'),
                              batch_size= batch_size_train)

In [ ]:

model2 = Sequential()
model2.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))


model2.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))

#model2.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME', activation='relu'))
#model2.add(MaxPooling2D((2, 2)))

#model2.add(Conv2D(filters=512, kernel_size=(3,3), padding='SAME', activation='relu'))
#model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())
model2.add(Dense(len(os.listdir('../own_dataset/diferent_dice/train')), activation='softmax'))
model2.summary()

In [ ]:
model2.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history2= model2.fit_generator(train_batches, steps_per_epoch= len(train_batches) , 
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=10)

In [ ]:
history2= model2.fit_generator(train_batches, steps_per_epoch= len(train_batches) , 
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=3)

In [ ]:
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=os.getcwd(),save_weights_only=True,verbose=1)

In [ ]:
model2.save('model_difdices_final.h5')

In [ ]:
model2_json = model2.to_json()
with open("model_difdices_final.json", "w") as json_file:
    json_file.write(model2_json)